In [142]:
from binance.client import Client
import pandas as pd 
import numpy as np 
from binance.enums import *
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import plotly_express as px
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import mpl_finance
from mpl_finance import candlestick2_ohlc
from mpl_finance import candlestick_ohlc
from sympy.stats import P, E, variance, Die, Normal
import time
from sklearn.naive_bayes import GaussianNB
import numpy as np

# Данная программа - торговый терминал для биржи Binance. Для чего она нужна? В теории для того, чтобы соединить множество
# бирж в одном месте, однако в связи с однотипностью задачи(подключения других бирж) в рамках проекта это не осуществлено. 
# Также данная программа может быть нужна для кастомизации биржи и добавлении новых возможностей, например таких как индиви-
# дуальные индикаторы, графики, виды ордеров или подключение ботов, анализ торговых стратегий. Многое из этого реализовано и может
# быть протестированно. Работает в данный 
# момент все с биржей Binance. Для того, чтобы хоть что-то заработало, необходимо создать Api на бирже. Для теста можно
# воспользоватьчся следующими ключами: api_key = "FkMzQ1yXzV2Cb4nROTnhIVq0zuPSLKqpiHYSWty5ifCI0GOD3HNCZiupfefYebAX"
# api_secret = "ha0wYkWYeJt4luU5YOGxIpuEhWNlsbODIBbB8FHVIMwDcJNAdz38WOcLcf0OUmfr" 
# Анализ вывода (варинат 3) в первый раз происходит очень долго, поэтому перед тем как его тестировать рекомендуется, закомен-
# тить указанную строку и запустить ее в другой ячейке. Строка (VeryImportantDataFrame = pd.DataFrame()) и запустить команду во
# во второй ячейке, предварительно скомпелировав 1-ую, так как в ней определена функция.
# pip install plotly
# pip install https://github.com/matplotlib/mpl_finance/archive/master.zip
#
#'ETH', 'LTC', 'BNB', 'NEO', 'QTUM', 'EOS', 'SNT', 'BNT', 'BCC', 'GAS', 'BTC', 'HSR', 'OAX', 'DNT', 'MCO', 'ICN', 'WTC', 'LRC', 'YOYO', 'OMG', 'ZRX', 'STRAT', 'SNGLS', 'BQX', 'KNC', 'FUN', 'SNM', 'IOTA', 'LINK', 'XVG', 'SALT', 'MDA', 'MTL', 'SUB', 'ETC', 'MTH', 'ENG', 'ZEC', 'AST', 'DASH', 'BTG', 'EVX', 'REQ', 'VIB', 'TRX', 'POWR', 'ARK', 'XRP', 'MOD', 'ENJ', 'STORJ', 'VEN', 'KMD', 'NULS', 'RCN', 'RDN', 'XMR', 'DLT', 'AMB', 'BAT', 'BCPT', 'ARN', 'GVT', 'CDT', 'GXS', 'POE', 'QSP', 'BTS', 'XZC', 'LSK', 'TNT', 'FUEL', 'MANA', 'BCD', 'DGD', 'ADX', 'ADA', 'PPT', 'CMT', 'XLM', 'CND', 'LEND', 'WABI', 'TNB', 'WAVES', 'GTO', 'ICX', 'OST', 'ELF', 'AION', 'NEBL', 'BRD', 'EDO', 'WINGS', 'NAV', 'LUN', 'TRIG', 'APPC', 'VIBE', 'RLC', 'INS', 'PIVX', 'IOST', 'CHAT', 'STEEM', 'NANO', 'VIA', 'BLZ', 'AE', 'RPX', 'NCASH', 'POA', 'ZIL', 'ONT', 'STORM', 'XEM', 'WAN', 'WPR', 'QLC', 'SYS', 'GRS', 'CLOAK', 'GNT', 'LOOM', 'BCN', 'REP', 'TUSD', 'ZEN', 'SKY', 'CVC', 'THETA', 'IOTX', 'QKC', 'AGI', 'NXS', 'DATA', 'SC', 'NPXS', 'KEY', 'NAS', 'MFT', 'DENT', 'ARDR', 'HOT', 'VET', 'DOCK', 'POLY', 'PHX', 'HC', 'GO', 'PAX', 'RVN', 'DCR', 'USDC', 'MITH', 'BCHABC', 'BCHSV', 'REN', 'BTT', 'USDS', 'ONG', 'FET', 'CELR'
# Все тикеры, которые торгуются на данной бирже

# Иногда(очень редко) по неизвестным причинам, программа не запускаетмя, нужно просто нажать на stop и снова запустить ее.
########################################### Variables ###################################################

api_key = "FkMzQ1yXzV2Cb4nROTnhIVq0zuPSLKqpiHYSWty5ifCI0GOD3HNCZiupfefYebAX" #дефолтный API ключ для аккаунта на бирже
api_secret = "ha0wYkWYeJt4luU5YOGxIpuEhWNlsbODIBbB8FHVIMwDcJNAdz38WOcLcf0OUmfr" 
#api_key = input("Enter API key", ) #Если вы хотите подключить свой аккаунт, то раскомментите данные функции, и законментите 
#api_secret = input("Enter API secret key", ) # значения которые выше

client = Client(api_key, api_secret)
# подключаемся к аккаунту на бирже и синхронизируем время
time_res = client.get_server_time()

VeryImportantDataFrame = pd.DataFrame() # законментировать данную строчку (описано в каком случае вверху) 

########################################### Functions ###################################################
# данная функция выводит все торугемые на бирже пары в виде словаря 
def get_all_listed_pairs():
    exchange_info = client.get_exchange_info()
    pairs = {}
    #print(exchange_info)
    for dictionary in exchange_info['symbols']:
        pairs[dictionary['baseAsset']] = []
    for dictionary in exchange_info['symbols']:
        pairs[dictionary['baseAsset']].append(dictionary['quoteAsset'])
    return pairs 

# данная фунция взаимодействует с пользователем. Вызывает все остальные функции в зависимости от выбора.
def Welcome_Function():
    print("""\n \n \n Choose what you want to do: \n 
              1) place an order \n 
              2) get last price \n
              3) learn how to withdraw in a best way \n 
              4) show chart \n
              5) stop programme \n
              6) show candelstick graph with bolliger bands \n
              7) Проверка гипотезы о Moving Average""" )
    print("To choose one of these options type 1 ... 7 accordingly")
    option = int(input("Число, Например(1)"))
    if (option == 1): 
        place_order()
    elif (option == 2): 
        tiker = input("Введите тикер: Например(ETH, BTC, XRP)", )
        price(tiker)
    elif (option == 3): 
        global VeryImportantDataFrame
        if  VeryImportantDataFrame.empty: 
            print("Programme is parsing info for the first time. This will take around 10 min")
            VeryImportantDataFrame = create_withdrowal_dataframe()
        best_withdrowal(df = VeryImportantDataFrame.copy(deep=True) )
    elif (option == 4):        
        print("This action due to plotly will stop the programme after execution.")
        ticker = input("Введите тикер: Например(ETH, BTC, XRP) ", )
        print("Доступные пары для торгов ")
        print(*pairs[ticker])
        pair = input("Введите пару из вышеперечисленный. Например(USDT)", )
        return show_graphics(ticker + pair)
    elif (option == 5):
        return
    elif (option == 6):
        candelstickGraphics()
    elif (option == 7):
        CheckIdea()
    else:
        print("Пожалуйста введите корректные данные")
        
# функция отвечающая за установку ордера. На пустом аккаунте (без денег) будет выдавать недостаточно денег на балансе. 
# я проверял на аккаунте с деньгами, ордера ставятся.
def place_order():
    token1 = str(input("Введите тикер: Например(ETH, BTC, XRP) ",))
    if token1 in pairs.keys():
        print("Доступные пары для торгов ")
        print(*pairs[token1])
        token2 = str(input("Введите пару из вышеперечисленный. Например(USDT)"),)
        if token2 in pairs[token1]:
            side = input("BUY or SELL: Написать либо BUY либо SELL", )
            if not(side in ["BUY", "SELL"]): 
                return
            print()
            get_5minavg_price(token1+token2)
            print("Ваш баланс: " + (client.get_asset_balance(asset=token1)["free"]) + " " + token1)
            print("Ваш баланс: " + (client.get_asset_balance(asset=token2)["free"]) + " " + token2)
            Your_Price = input("Цена: ",)
            Your_Quantity = input("Колличество: ", )
            #balance check ?
            if ( ((float(Your_Quantity) > float(client.get_asset_balance(asset=token1)["free"])) and (side == "SELL")) or ((float(Your_Quantity) * float(Your_Price) >= float(client.get_asset_balance(asset=token2)["free"]) ) and (side=="BUY"))):
                print("Недостаточно денег на балансе")
                return
            order = client.create_order(
                symbol= str(token1 + token2),
                side=side,
                type=ORDER_TYPE_LIMIT,
                timeInForce=TIME_IN_FORCE_GTC,
                quantity=Your_Quantity,
                price=str(Your_Price))
            #orders = client.get_open_orders(symbol=str(token1 + token2))
            #print(orders)
            print("Ордер размещен")
    else:
        print("Не существует такой торгующейся пары")
        

        
# выводит среднюю цену за последние 5 минут       
def get_5minavg_price(tiker):
    URL  = "https://api.binance.com/api/v3/avgPrice" 
    params = {
            "symbol": tiker
    }
    q = requests.get(URL , params= params)
    print(q.text)
    
    
# парсит страницу биржи, создает таблицу с характеристиками вывода для всех монет торгующихся на бирже. Очень долго создается
# так как приводит все коммисии биржи в долларовое выражение. Это более 200 запросов к бирже на счет текущих котировок. 
def create_withdrowal_dataframe():#(tiker, amount)
    URL = "https://www.binance.com/en/fee/schedule"
    r = requests.get(URL)
    page = BeautifulSoup(r.text, "html.parser")
    WithdrowalFee = []
    MinWithdrowal = []
    AssetArray = []
    for token in page.findAll("tr"):
        if len(token.findAll("span"))>2:
            Fee = str(token.findAll("span")[2])
            Min = str(token.findAll("span")[1])
            #print(str(token.findAll("span")[2]))
            try:
                float(Fee[Fee.find(">") + 1:Fee.find("<",2)])
                float(Min[Min.find(">") + 1:Min.find("<",2)])
                AssetArray.append(str(Fee[findn(Fee,">",2) + 1:Fee.find("<",findn(Fee,">",2))]))
                WithdrowalFee.append(float(Fee[Fee.find(">") + 1:Fee.find("<",Fee.find(">"))]))
                MinWithdrowal.append(float(Min[Min.find(">") + 1:Min.find("<",Min.find(">"))]))
            except:
                continue
    #print(AssetArray)
    dictionary ={"Minimum Withdrawal" : MinWithdrowal , "Withdrawal Fee" :WithdrowalFee , "Asset" : AssetArray }
    WithdrawDF = pd.DataFrame(dictionary)
    WithdrawDF["Asset Price"] = WithdrawDF["Asset"]
    print("""\n \n Creating withdrowal dataframe \n \n """)
    print(WithdrawDF)
    print("""\n \n Parsing actual USD prices \n \n """)
    WithdrawDF["Asset Price"] = WithdrawDF["Asset Price"].apply(lambda x : price(x))
    #get_5minavg_price(WithdrawDF["Asset"])
    #for ticker in Coin_list:
     #   fees[ticker] = client.get_withdraw_fee(asset =ticker) * get_5minavg_price(ticker)
     #   fees_df = pd.Dataframe(data = fees)
    #return fees_df
    return WithdrawDF
#WithdrawDF = create_withdrowal_dataframe()

# функция из интернета отвечающая за поиск n-того вхождения элемента
#FROM: https://www.w3schools.com/python/ref_string_find.asp
def findn(haystack, needle, n):
    parts= haystack.split(needle, n+1)
    if len(parts)<=n+1:
        return -1
    return len(haystack)-len(parts[-1])-len(needle)
#string = "<bad><good>"
#print(string[findn(string,"<",1)+1:findn(string,">",1)])
#END FROM: https://www.w3schools.com/python/ref_string_find.asp

# функция находящяя долларовою цену по введенному тикеру
def price(tiker):
    try:
        URL  = "https://api.binance.com/api/v3/avgPrice" 
        params = {
                "symbol": tiker + "BTC"
        }
        q = requests.get(URL , params= params)
        BTCprice = float(q.text[ q.text.find("\"price\"") +9: len(q.text)-2])
        URL  = "https://api.binance.com/api/v3/avgPrice" 
        params = {
                "symbol": "BTCUSDT"
        }
        q = requests.get(URL , params= params)
        USDTprice  = float(q.text[ q.text.find("\"price\"") +9: len(q.text)-2]) * BTCprice
        print(str(tiker) + " " + str(USDTprice)+ " USDT")
        return USDTprice
    except:
        try: 
            URL  = "https://api.binance.com/api/v3/avgPrice" 
            params = {
                    "symbol": tiker + "ETH"
            }
            q = requests.get(URL , params= params)
            BTCprice = float(q.text[ q.text.find("\"price\"") +9: len(q.text)-2])
            URL  = "https://api.binance.com/api/v3/avgPrice" 
            params = {
                    "symbol": "ETHUSDT"
            }
            q = requests.get(URL , params= params)
            USDTprice  = float(q.text[ q.text.find("\"price\"") +9: len(q.text)-2]) * BTCprice
            print(str(tiker) + " " + str(USDTprice)+ " USDT")
            return USDTprice
        except: 
            try: 
                URL  = "https://api.binance.com/api/v3/avgPrice" 
                params = {
                        "symbol": tiker + "USDT"
                }
                q = requests.get(URL , params= params)
                USDTprice  = float(q.text[ q.text.find("\"price\"") +9: len(q.text)-2])
                print(str(tiker) + " " + str(USDTprice) + " USDT")
                return USDTprice
            except: 
                return -1

# функция взаимодействующая с таблицей по выводам, рассчитывает лучшую возможность для вывода средст
# учитывая текущих курс, размер вывода, валюту вывода и тд.
# Советую попробовать вывести 1 доллар в битках и 100000 долларов в битках, чтобы увидеть разницу. Коммисия за вывод
#показывается в долларах.
def best_withdrowal(df):
    tokenName = input("Введите, что вы хотите вывести. Например(BTC, ETH, XRP): ", )
    Value = input("Введите сумму в долларах, которую хотите вывеси. Например(1000): ", )
    df["Asset fee"] = df["Asset Price"] * df["Withdrawal Fee"]
    df["Minimum Withdrawal(USD)"] = df["Asset Price"] * df["Minimum Withdrawal"]
    df.drop(df[df["Asset Price"]==-1].index, axis=0, inplace =True)
    df.drop(df[df["Minimum Withdrawal(USD)"] > float(Value)].index , axis = 0, inplace = True)
    df["Asset fee"] = df["Asset fee"] + 0.001 *float(Value)
    df.loc[ df["Asset"] == tokenName, "Asset fee"] -= 0.001 *float(Value)
    #print(df)
    print("Вы можете вывести в: ")
    print(*df["Asset"])
    print("Минимальная коммисия будет в: " + df.loc[df["Asset fee"] == df["Asset fee"].min()]["Asset"].astype(str) + " " +  df["Asset fee"].min().astype(str))
    #print("You can withdraw minimum amount with: " + df.loc[df["Minimum Withdrawal(USD)"] == df["Minimum Withdrawal(USD)"].min()]["Asset"].astype(str) + " " + df["Minimum Withdrawal(USD)"].min().astype(str))

# показывает интерактивных график последних 500 сделок(ограниченеи API) по заданнмоу тикеру. По осям соотвественно цена и время,
# а размер точки, это объем данной сделки. ( что очень интуитивно для понимания сути движения цены)
def show_graphics(ticker):
    trades = client.get_recent_trades(symbol=ticker)
    prices = []
    time = []
    volume = []
    for dictionary in trades:
        prices.append(dictionary["price"])
        time.append((datetime.datetime.fromtimestamp(dictionary["time"]/1000))) #[11:19]
        volume.append(float(dictionary["qty"]))
    dc = {"time" :time,"prices": prices , "volume" : volume}    
    df = pd.DataFrame(dc)
    #px.scatter(df , x = "time" , y = "prices" , size = "volume")
    #config = {'scrollZoom': True}
    return px.scatter(df , x = "time" , y = "prices" , size = "volume" )


# функция выводит график в виде японских свечей, также на первом графике рассчитан популярный индикатор Bollinger Bands,
# который представляет из себя скользящуу среднюю(средняя линия), а также верхняя(нижняя) линии это значние скользящей средней
# плюс(минус) два стандартных отклонения за определенный период.
# Нижний графи просто выводит японские свечи не заграможденные индикаторами.
def candelstickGraphics():
    
    ticker = input("Введите тикер: Например(ETH, BTC, XRP): ", )
    #ticker = "ETH"
    KLINE_INTERVAL = input("""\n \n Введите время свечи. Например(5m , 15m, 1h, 1d) \n \n """)
    #KLINE_INTERVAL = "1h"
    print("Доступные пары для торгов ")
    print(*pairs[ticker])
    pair = input("Введите пару из вышеперечисленный. Например(USDT)", )
    #pair = "USDT"
    period = input("""Введите период: Например(7 day ago UTC, 1 Jan, 2018 ) \n введите одну дату, если хотите увидеть 
    график от данной даты до настоящего момента,  или введите две даты через символ | чтобы увидеть график между ними \n """, )
    #period = "7 day ago UTC"
    twoperiods =[]
    if "|" in period:
        twoperiods = preiod.split("|")
        if KLINE_INTERVAL == "5m":
            klines = client.get_historical_klines(ticker + pair, Client.KLINE_INTERVAL_5MINUTE, twoperiods[0], twoperiods[1])
        elif KLINE_INTERVAL == "15m":
            klines = client.get_historical_klines(ticker + pair, Client.KLINE_INTERVAL_15MINUTE, twoperiods[0], twoperiods[1])
        elif KLINE_INTERVAL == "1h": 
            klines = client.get_historical_klines(ticker + pair, Client.KLINE_INTERVAL_1HOUR, twoperiods[0], twoperiods[1])
        elif KLINE_INTERVAL == "1d": 
            klines = client.get_historical_klines(ticker + pair, Client.KLINE_INTERVAL_1DAY, twoperiods[0], twoperiods[1])
        else:
            print("Вы ввели недопустимый интервал")
            return
    else:
        if KLINE_INTERVAL == "5m":
            klines = client.get_historical_klines(ticker + pair, Client.KLINE_INTERVAL_5MINUTE, period)
        elif KLINE_INTERVAL == "15m":
            klines = client.get_historical_klines(ticker + pair, Client.KLINE_INTERVAL_15MINUTE, period)
        elif KLINE_INTERVAL == "1h": 
            klines = client.get_historical_klines(ticker + pair, Client.KLINE_INTERVAL_1HOUR, period)
        elif KLINE_INTERVAL == "1d": 
            klines = client.get_historical_klines(ticker + pair, Client.KLINE_INTERVAL_1DAY, period)
        else:
            print("Вы ввели недопустимый интервал")
            return
    #print(klines)
    int_kliens = []
    for kline in klines:
        line =[]
        for number in kline:
            line.append(float(number))
        int_kliens.append(line)
    #print(int_kliens)
    asset_dates = []
    asset_open =[]
    asset_close = []
    asset_high = []
    asset_low = []
    moving_average = [0] * 19
    for line in int_kliens: 
        asset_dates.append(datetime.datetime.fromtimestamp(line[0]/1000))
        asset_open.append(line[1])
        asset_high.append(line[2])
        asset_low.append(line[3])
        asset_close.append(line[4])
    j = 0
    while j+20 < len(asset_close):
        i = 0
        average = 0
        for i in range(20):
            average += asset_close[i+j]
        average = average/20
        j +=1
        moving_average.append(average)
    #print(moving_average)  
    moving_average1 = []
    moving_average2 = []
    moving_average1.append(moving_average[i] *1.1 for i in range(len(moving_average)))
    moving_average2.append(moving_average[i] *0.9 for i in range(len(moving_average)))
    d = {"Date" : asset_dates , "Open" : asset_open , "High": asset_high , "Low" : asset_low, "Close" : asset_close}
    df = pd.DataFrame(d)
    df['Date'] = pd.to_datetime(df['Date'])
    df["Date"] = df["Date"].apply(mdates.date2num)
    ohlc= df[['Date', 'Open', 'High', 'Low','Close']].copy()  
    f1, figure1 = plt.subplots(figsize = (20,10)) 
    figure = plt.figure(figsize = (20,10))
    graph = figure.add_subplot(1,1,1)
    candlestick_ohlc(figure1, ohlc.values, width=20/(len(moving_average)*4), colorup='green', colordown='red')  #https://www.techtrekking.com/how-to-plot-simple-and-candlestick-chart-using-python-pandas-matplotlib/
    figure1.xaxis.set_major_formatter(mdates.DateFormatter('%d-%h'))
    ohlc['MA20'] = ohlc["Close"].rolling(20).mean()
    #ohlc["Var"] = ohlc["Close"].rolling(20).var()
    ohlc["STD_Dev"] = ohlc["Close"].rolling(20).std()
    ohlc["Up_Channel"] =  ohlc['MA20'] + 2 *ohlc["STD_Dev"]
    ohlc["Down_Channel"] =  ohlc['MA20'] - 2 *ohlc["STD_Dev"]
    figure1.plot(ohlc['Date'], ohlc['MA20'], color = 'orange', label = 'MA20')
    figure1.plot(ohlc['Date'], ohlc['Up_Channel'], color = 'blue', label = 'Up')
    figure1.plot(ohlc['Date'], ohlc['Down_Channel'], color = 'blue', label = 'Down')
    
    candlestick2_ohlc(graph,asset_open ,asset_high , asset_low ,asset_close , width=0.4, colorup='green', colordown='purple')
    #graph.plot()
    time.sleep(10)
    
    
    
# данная функция создает таблицу необходимую для функции CheckIdea(). В данную таблицу включаются цены закрытия японских свеч 
# определенного размера и за определенный период.
def createDataFrame():
    ticker = input("Введите тикер: Например(ETH, BTC, XRP): ", )
    #ticker = "BTC"
    KLINE_INTERVAL = input("""\n \n Введите время свечи. Например(5m , 15m, 1h, 1d) \n \n """)
    #KLINE_INTERVAL = "1d"
    print("Доступные пары для торгов ")
    print(*pairs[ticker])
    pair = input("Введите пару из вышеперечисленный. Например(USDT)", )
    #pair = "USDT"
    if KLINE_INTERVAL == "1h":
        period = "40 day ago UTC"
        klines = client.get_historical_klines(ticker + pair, Client.KLINE_INTERVAL_1HOUR, period)
    elif KLINE_INTERVAL == "1d":
        period = "900 day ago UTC"
        klines = client.get_historical_klines(ticker + pair, Client.KLINE_INTERVAL_1DAY, period)
    else: 
        print("Введите правильное значение")
        return
    int_kliens = []
    for kline in klines:
        line =[]
        for number in kline:
            line.append(float(number))
        int_kliens.append(line)
    asset_dates = []
    asset_close = []
    for line in int_kliens: 
        asset_dates.append(datetime.datetime.fromtimestamp(line[0]/1000))
        asset_close.append(line[4])
    d = {"Date" : asset_dates , "Close" : asset_close}
    df = pd.DataFrame(d)
    return df
     
    
# Идея использования модели взята с данного ресурса: #https://www.analyticsvidhya.com/blog/2017/09/naive-bayes-explained/   
# Данная функция использует машинное обучение в виде наивного Баесовского критерия. Она тестирует ниже описанных факт на историчесих
# данных. Алгоритм следующйи: получаем цены закрытия свеч за определенный период. Далее находим указанные скользящие средние, далее 
# находим разницу между более и менее долгорочными скользящими средними. В итоге получаем точки пересечения данных прямых, при чем 
# записываем не только сам факт их присутствия, но и то в какую сторону было пересечение. То есть короткая MA пересекла длинную 
# MA сверху или снизу. Далее сравниваем полученные точки с реаьным изменением цены в течении следующего дня и следующей недели. 
# Модель далее учится на описанных данных и выдает результат на самом ли деле, описанные пересечения приводят к какому то опре-
# деленному движению цен. Протестировав данную модель на числахз получил ожидаемых результат, что чем более долгосрочные средние 
# пересекаются, тем достовернее данный факт. Рекомендую проверить на 5 и 20 дневных средних, на графике BTC например.

def CheckIdea():
    df = createDataFrame()
    print("""Существует распространенная модель, что пересечение более краткосрочной скользящей средней более долгосроную, 
    ведет к движению в сторону пересечения. То есть если 20 дневная скользящая пересекла 200 дневную снизу вверх, то движение
    будет вверх. Данная функция использует наивный Баесовский критерий, чтобы обучаясь на историчесикх данных предсказывать, так ли это. 
    Можно выбрать тест как на часовых свечах, так и на дневных.(Красивые результаты получаются, если использовать 5,20 дневную среднюю)
    """)
    SRMA = input("Введите более краткосрочную MA(Moving Average)", )
    LRMA = input("Введите более долгосрочную MA(Moving Average)", )
    df["SRMA"] = df["Close"].rolling(int(SRMA)).mean()
    df["LRMA"] = df["Close"].rolling(int(LRMA)).mean()
    df["Difference"] = round(df["LRMA"] - df["SRMA"])
    df["Interisting Point"] = 0
    df["Real Price Change 1d"] = 0
    df["Real Price Change 7d"] = 0
    #zerodf = df.drop(df[df["Difference"] != 0].index )
    for index, row in df.iterrows():
        if index != 0:
            if (df.loc[index, "Difference"] * df.loc[index - 1, "Difference"]) < 0:
                #print((df.loc[index, "Difference"] * df.loc[index - 1, "Difference"]))
                if df.loc[index - 1, "Difference"] < 0:
                    df.loc[index , "Interisting Point"] =  1 #"Exp(UP)"
                    try:
                        df.loc[index , "Real Price Change 1d"] = int(df.loc[index + 1 , 'Close']  - df.loc[index , 'Close']) 
                        df.loc[index , "Real Price Change 7d"] = int(df.loc[index + 7 , 'Close']  - df.loc[index , 'Close']) 
                    except:
                        continue
                else: 
                    df.loc[index , "Interisting Point"] =  -1 #"Exp(DOWN)"
                    try:
                        df.loc[index , "Real Price Change 1d"] = int(df.loc[index + 1 , 'Close']  - df.loc[index , 'Close']) 
                        df.loc[index , "Real Price Change 7d"] = int(df.loc[index + 7 , 'Close']  - df.loc[index , 'Close']) 
                    except:
                        continue
    ndf = df.drop(df[df["Real Price Change 7d"] == 0].index )

    #print(df)
    #print(ndf)
    InterestingPoints  = ndf["Interisting Point"].tolist()
    #print(InterestingPoints)
    Change1 = ndf["Real Price Change 1d"].tolist()
    #print(Change1)
    Change7 = ndf["Real Price Change 7d"].tolist()
    #print(Change7)
    z = []
    for element in InterestingPoints:
        row = [1] * 1
        row.append(element)
        z.append(row)
    #FROM https://www.analyticsvidhya.com/blog/2017/09/naive-bayes-explained/
    x = np.array(z)
    y = np.array(Change1)

    #print(len(z))
    #print(len(y))
    #Create a Gaussian Classifier
    model = GaussianNB()

    # Train the model using the training sets 
    model.fit(x, y)
    #END FROM
    #Predict Output 
    print('Если из пересечения линий следует снижение, то модель предсказывает: ' + str(model.predict([[1,-1]])) + ' долларов движения')
    print('Если из пересечения линий следует рост, то модель предсказывает: ' + str(model.predict([[1,1]])) + ' долларов движения')

    print("""Как интерпретировать полученный результат? Данная модель использует простейшее машинное обучение и основывая на исторических
    данных предсказывает, как будет вести себя цена в будущем. Например, если ожидалось движение вниз, то отрицательное ожидаемое число будет
    подтверждать данное наблюдение. Однако, не стоит обращать внимание на числа, так как очевидно такие сигналы ничего не говорят о силе 
    движения, поэтому усовершествовать модель в данном направлении не вижу смысла.
    """)


##############################################################################################
# запуск вышеописанных функций
pairs = get_all_listed_pairs()

Welcome_Function()






 
 
 Choose what you want to do: 
 
              1) place an order 
 
              2) get last price 

              3) learn how to withdraw in a best way 
 
              4) show chart 

              5) stop programme 

              6) show candelstick graph with bolliger bands 

              7) Проверка гипотезы о Moving Average
To choose one of these options type 1 ... 7 accordingly
Число, Например(1)7
Введите тикер: Например(ETH, BTC, XRP): BTC

 
 Введите время свечи. Например(5m , 15m, 1h, 1d) 
 
 1d
Доступные пары для торгов 
USDT TUSD PAX USDC USDS
Введите пару из вышеперечисленный. Например(USDT)USDT
Существует распространенная модель, что пересечение более краткосрочной скользящей средней более долгосроную, 
    ведет к движению в сторону пересечения. То есть если 20 дневная скользящая пересекла 200 дневную снизу вверх, то движение
    будет вверх. Данная функция использует наивный Баесовский критерий, чтобы обучаясь на историчесикх данных предсказывать, так ли это. 
 

VeryImportantDataFrame = pd.DataFrame()

In [24]:
#VeryImportantDataFrame = create_withdrowal_dataframe() Данную строчку для проверки рекомендуется разкоментить 
# в замен той что на верху. Так будет быстрее, так как данная таблица будет парситься 1 раз, а не каждый раз как выби-
# -рается 3-яя опция.


 
 Creating withdrowal dataframe 
 
 
     Minimum Withdrawal  Withdrawal Fee   Asset Asset Price
0                 0.120          0.0600     BNB         BNB
1                 0.002          0.0005     BTC         BTC
2                 1.000          0.0000     NEO         NEO
3                 0.020          0.0100     ETH         ETH
4                 0.002          0.0010     LTC         LTC
5                 0.020          0.0100    QTUM        QTUM
6                 0.200          0.1000     EOS         EOS
7                76.000         38.0000     SNT         SNT
8                 3.000          1.5000     BNT         BNT
9                 0.001          0.0000     GAS         GAS
10               10.000          5.0000     BTM         BTM
11                8.400          4.2000    USDT        USDT
12                0.001          0.0005     HCC         HCC
13                8.400          4.2000     OAX         OAX
14              106.000         53.0000     DNT         DNT
1

TRIG 0.10116097377507 USDT
APPC 0.0820016984388825 USDT
PIVX 0.970839614004486 USDT
